<a href="https://colab.research.google.com/github/rahultrada/National_DS_Bowl/blob/main/ensemble_preds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/Saved models/train.zip'

Streaming output truncated to the last 5000 lines.
  inflating: train/trichodesmium_puff/110182.jpg  
  inflating: train/trichodesmium_puff/110212.jpg  
  inflating: train/trichodesmium_puff/110253.jpg  
  inflating: train/trichodesmium_puff/110463.jpg  
  inflating: train/trichodesmium_puff/110493.jpg  
  inflating: train/trichodesmium_puff/110545.jpg  
  inflating: train/trichodesmium_puff/110562.jpg  
  inflating: train/trichodesmium_puff/11076.jpg  
  inflating: train/trichodesmium_puff/110777.jpg  
  inflating: train/trichodesmium_puff/110808.jpg  
  inflating: train/trichodesmium_puff/110836.jpg  
  inflating: train/trichodesmium_puff/111004.jpg  
  inflating: train/trichodesmium_puff/111043.jpg  
  inflating: train/trichodesmium_puff/111178.jpg  
  inflating: train/trichodesmium_puff/111289.jpg  
  inflating: train/trichodesmium_puff/111345.jpg  
  inflating: train/trichodesmium_puff/111417.jpg  
  inflating: train/trichodesmium_puff/111435.jpg  
  inflating: train/trichodesmium

In [ ]:
#Invert image color
from PIL import Image, ImageOps
import torchvision.transforms.functional as FF


class Invert(object):
    """Inverts the color channels of an PIL Image
    while leaving intact the alpha channel.
    """
    
    def invert(self, img):
        r"""Invert the input PIL Image.
        Args:
            img (PIL Image): Image to be inverted.
        Returns:
            PIL Image: Inverted image.
        """
        if not FF._is_pil_image(img):
            raise TypeError('img should be PIL Image. Got {}'.format(type(img)))

        if img.mode == 'RGBA':
            r, g, b, a = img.split()
            rgb = Image.merge('RGB', (r, g, b))
            inv = ImageOps.invert(rgb)
            r, g, b = inv.split()
            inv = Image.merge('RGBA', (r, g, b, a))
        elif img.mode == 'LA':
            l, a = img.split()
            l = ImageOps.invert(l)
            inv = Image.merge('LA', (l, a))
        else:
            inv = ImageOps.invert(img)
        return inv

    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to be inverted.
        Returns:
            PIL Image: Inverted image.
        """
        return self.invert(img)

    def __repr__(self):
        return self.__class__.__name__ + '()'

In [ ]:
data_dir = '/content/train'
def load_split_train_valid(datadir, valid_size = .2, batchsize=512):
    mean, std =[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    train_transforms = transforms.Compose([transforms.Resize(256),
                                           transforms.CenterCrop(224),
                                           Invert(),
                                           transforms.RandomRotation(90),
                                           transforms.RandomVerticalFlip(0.5),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean,std)])
    valid_transforms = transforms.Compose([transforms.Resize(256),
                                           transforms.CenterCrop(224),
                                           Invert(),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean,std)])
    train_data = datasets.ImageFolder(datadir, transform=train_transforms)
    valid_data = datasets.ImageFolder(datadir, transform=valid_transforms)
    from torch.utils.data.sampler import SubsetRandomSampler
    from sklearn.model_selection import train_test_split
    targets = train_data.targets
    train_idx, valid_idx = train_test_split(np.arange(len(targets)), test_size=valid_size, random_state=42, shuffle=True, stratify=targets)
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batchsize)
    validloader = torch.utils.data.DataLoader(valid_data,
                   sampler=valid_sampler, batch_size=batchsize)
    return trainloader, validloader

trainloader, validloader = load_split_train_valid(data_dir, .1, 64)

In [ ]:
with open('/content/drive/My Drive/Saved models/indexes.txt') as f:
    indexes = f.read().splitlines()

In [ ]:
print(indexes)

['1.jpg', '10.jpg', '100.jpg', '1000.jpg', '10000.jpg', '100000.jpg', '100001.jpg', '100002.jpg', '100003.jpg', '100004.jpg', '100005.jpg', '100006.jpg', '100007.jpg', '100008.jpg', '10001.jpg', '100011.jpg', '100012.jpg', '100013.jpg', '100014.jpg', '100015.jpg', '100016.jpg', '100017.jpg', '100018.jpg', '100019.jpg', '10002.jpg', '100020.jpg', '100021.jpg', '100022.jpg', '100023.jpg', '100024.jpg', '100025.jpg', '100026.jpg', '100027.jpg', '100028.jpg', '10003.jpg', '100030.jpg', '100031.jpg', '100032.jpg', '100033.jpg', '100034.jpg', '100035.jpg', '100036.jpg', '100037.jpg', '100038.jpg', '100039.jpg', '10004.jpg', '100041.jpg', '100042.jpg', '100043.jpg', '100045.jpg', '100046.jpg', '100047.jpg', '100048.jpg', '100049.jpg', '10005.jpg', '100050.jpg', '100051.jpg', '100052.jpg', '100053.jpg', '100054.jpg', '100055.jpg', '100057.jpg', '100059.jpg', '10006.jpg', '100060.jpg', '100062.jpg', '100066.jpg', '100067.jpg', '100068.jpg', '100069.jpg', '10007.jpg', '100071.jpg', '100072.jpg',

In [ ]:
predictions = []

# predictions.append(np.loadtxt('/content/drive/My Drive/Saved models/predictions_resnet34.txt')) 
# predictions.append(np.loadtxt('/content/drive/My Drive/Saved models/predictions_resnet50_1.txt')) 
# predictions.append(np.loadtxt('/content/drive/My Drive/Saved models/predictions_resnet50_2.txt'))
# predictions.append(np.loadtxt('/content/drive/My Drive/Saved models/predictions_resnet152.txt'))

predictions.append(0.1 * np.loadtxt('/content/drive/My Drive/Saved models/predictions_densenet169_1.txt')) 
predictions.append(0.1 * np.loadtxt('/content/drive/My Drive/Saved models/predictions_densenet169_2.txt')) 
predictions.append(0.1 * np.loadtxt('/content/drive/My Drive/Saved models/predictions_densenet169_3.txt')) 
predictions.append(0.05 * np.loadtxt('/content/drive/My Drive/Saved models/predictions_densenet169_4.txt')) 
predictions.append(0.05 * np.loadtxt('/content/drive/My Drive/Saved models/predictions_densenet169_5.txt')) 
predictions.append(0.3 * np.loadtxt('/content/drive/My Drive/Saved models/predictions_densenet169_6.txt')) 
predictions.append(0.3 * np.loadtxt('/content/drive/My Drive/Saved models/predictions_densenet169_7.txt')) 


In [ ]:
mean_models_test = np.sum( np.array(predictions), axis=0 )

In [ ]:
mean_models_test.shape

(130400, 121)

In [ ]:
import pandas as pd
df = pd.DataFrame(mean_models_test, columns=trainloader.dataset.classes, index=indexes)
df.index.name = 'image'
df.to_csv('submissionmean.csv')

In [ ]:
!zip submissionmean.zip submissionmean.csv

  adding: submissionmean.csv (deflated 57%)
